In [9]:
import os
os.environ['OPEN_AI_KEY'] = os.getenv('OPEN_AI_KEY')
from langchain import OpenAI
from dotenv import load_dotenv
from llama_index import LLMPredictor, ServiceContext
load_dotenv()

True

In [6]:
os.environ['OPEN_AI_KEY'] = os.getenv('OPEN_AI_KEY')

In [10]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.6, api_key=os.environ['OPEN_AI_KEY'], model_name="gpt-3.5-turbo"))

c:\Users\emmau\Downloads\chatbot\chatbot\lib\site-packages\langchain_community\llms\openai.py:248: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\emmau\Downloads\chatbot\chatbot\lib\site-packages\langchain_community\llms\openai.py:1057: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [11]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex
document = SimpleDirectoryReader('data').load_data()

In [12]:
document

[Document(id_='58c46425-51dc-4c58-9879-0f48ebb73aa5', embedding=None, metadata={'page_label': 'Page 1', 'file_name': 'AGO_DECEMBER_2023_REPORT.pdf', 'file_path': 'data\\AGO_DECEMBER_2023_REPORT.pdf', 'file_type': 'application/pdf', 'file_size': 616139, 'creation_date': '2024-02-18', 'last_modified_date': '2024-02-18', 'last_accessed_date': '2024-02-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='(DECEMBER 2023)  Automotive Gas Oil  \n(Diesel) Price Watch  \nReport Date: January 2024  ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='04101c24-4724-4896-a791-87eb425cf2c2', embedding=None, metadata={'page_label': 'Page 2', 'file_name': 'A

In [13]:
index = VectorStoreIndex.from_documents(document, show_progress=True)

c:\Users\emmau\Downloads\chatbot\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 241/241 [00:04<00:00, 49.01it/s]


In [31]:
index

In [32]:
query_engine = index.as_query_engine()

In [33]:
response = query_engine.query("Who was Gatsby in love with?")

In [34]:
response

Response(response='Daisy', source_nodes=[NodeWithScore(node=TextNode(id_='3a1ed968-6cea-49b7-9d52-28679083abdf', embedding=None, metadata={'page_label': '81', 'file_name': 'the-great-gatsby.pdf', 'file_path': 'data\\the-great-gatsby.pdf', 'file_type': 'application/pdf', 'file_size': 763582, 'creation_date': '2024-02-06', 'last_modified_date': '2024-02-02', 'last_accessed_date': '2024-02-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0747ffec-83d8-4cd0-aace-c483abbb79cf', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '81', 'file_name': 'the-great-gatsby.pdf', 'file_path': 'data\\the-great-gatsby.pdf', 'file_type': 'application/pdf', 'file_size': 763582, 'creation_date': '2024-02-06', '

In [35]:
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

In [3]:
from llama_index.memory import ChatMemoryBuffer

In [14]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

#llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.6, api_key=os.environ['OPEN_AI_KEY'], model_name="gpt-3.5-turbo"))
llm = OpenAI(model="gpt-3.5-turbo", api_key=os.environ['OPEN_AI_KEY'], temperature=0.6)
doc = SimpleDirectoryReader("data").load_data()

In [15]:
index = VectorStoreIndex.from_documents(doc, show_progress=True)

Generating embeddings: 100%|██████████| 241/241 [00:26<00:00,  9.03it/s]


In [16]:
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    llm=llm,
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about data related to Nigeria. "
    ),
)

In [20]:
response = chat_engine.chat("Give me the overall")
response

AgentChatResponse(response="I don't have access to real-time data, but based on the reports I have, the overall inflation rate in Nigeria can vary by region and time period. In the reports I referenced earlier, the overall inflation rates were provided for specific states and periods, such as December 2023 and January 2024. For the most current overall inflation rate in Nigeria, I recommend checking the latest reports from the National Bureau of Statistics or other official sources.", sources=[ToolOutput(content="system: You are a chatbot, able to have normal interactions, as well as talk about data related to Nogeria.\nContext information is below.\n--------------------\npage_label: Page 3\nfile_path: data\\AGO_DECEMBER_2023_REPORT.pdf\n\n3 METHODOLOGY  \nThe data collection was conducted in sample outlets spread across 774 local govern-\nment areas in all the 36 states and the FCT, Abuja. The data was collected from over \n10,000 respondents located in the outlets. The estimates were

In [21]:
print(response)

I don't have access to real-time data, but based on the reports I have, the overall inflation rate in Nigeria can vary by region and time period. In the reports I referenced earlier, the overall inflation rates were provided for specific states and periods, such as December 2023 and January 2024. For the most current overall inflation rate in Nigeria, I recommend checking the latest reports from the National Bureau of Statistics or other official sources.


In [22]:
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./store"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)